In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
imdb_data = keras.datasets.imdb

In [3]:
(train_data, train_labels), (test_data, test_labels) = imdb_data.load_data(num_words=10000)

In [4]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
word_index = imdb_data.get_word_index()

word_index = {k: (v + 3) for k, v in word_index.items()}
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<UNUSED>'] = 3

reverse_word_index = dict([(v, k) for (k, v) in word_index.items()])

In [6]:
def decode_review(encoded_text):
    return ' '.join(reverse_word_index.get(i, '?') for i in encoded_text)

In [7]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [8]:
MAX_REVIEW_LEN = 256
REVIEW_PADDING_STRING = '<PAD>'

In [9]:
train_data = keras.preprocessing.sequence.pad_sequences(
    train_data,
    padding='post',
    value=word_index[REVIEW_PADDING_STRING],
    maxlen=MAX_REVIEW_LEN
)

test_data = keras.preprocessing.sequence.pad_sequences(
    test_data,
    padding='post',
    value=word_index[REVIEW_PADDING_STRING],
    maxlen=MAX_REVIEW_LEN
)

In [10]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [11]:
model = keras.Sequential()

In [12]:
model.add(keras.layers.Embedding(10000, 32))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 321,089
Trainable params: 321,089
Non-trainable params: 0
_________________________________________________________________


In [15]:
def split_indices(n, val_pct=0.1, seed=0):
    n_val = int(val_pct * n)
    np.random.seed(seed)
    idxs = np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

In [16]:
train_idxs, val_idxs = split_indices(len(train_data), val_pct=0.3)

In [17]:
train_idxs[:5], val_idxs[:5]

(array([15705,  6016, 24764, 11962, 15824]),
 array([14149,  8946, 22378, 12162,  4879]))

In [18]:
x_train, x_val = train_data[train_idxs], train_data[val_idxs]

In [19]:
y_train, y_val = train_labels[train_idxs], train_labels[val_idxs]

In [20]:
fit_model = model.fit(x_train, y_train, epochs=50, batch_size=512, validation_data=(x_val, y_val), verbose=1, workers=-1)

Train on 17500 samples, validate on 7500 samples
Epoch 1/50
17500/17500 [==============================] - 3s 164us/sample - loss: 0.6905 - accuracy: 0.6037 - val_loss: 0.6848 - val_accuracy: 0.7263
Epoch 2/50
17500/17500 [==============================] - 1s 61us/sample - loss: 0.6725 - accuracy: 0.7309 - val_loss: 0.6555 - val_accuracy: 0.7620
Epoch 3/50
17500/17500 [==============================] - 1s 61us/sample - loss: 0.6241 - accuracy: 0.7893 - val_loss: 0.5931 - val_accuracy: 0.7887
Epoch 4/50
17500/17500 [==============================] - 1s 62us/sample - loss: 0.5439 - accuracy: 0.8244 - val_loss: 0.5112 - val_accuracy: 0.8193
Epoch 5/50
17500/17500 [==============================] - 1s 60us/sample - loss: 0.4561 - accuracy: 0.8533 - val_loss: 0.4386 - val_accuracy: 0.8415
Epoch 6/50
17500/17500 [==============================] - 1s 61us/sample - loss: 0.3832 - accuracy: 0.8747 - val_loss: 0.3885 - val_accuracy: 0.8496
Epoch 7/50
17500/17500 [==============================] 

In [21]:
res = model.evaluate(test_data, test_labels)

25000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [22]:
res

[0.5624149834787846, 0.85424]

In [23]:
test_review = test_data[10]
prediction = model.predict([test_review])
print('Review:', decode_review(test_review))
print('Prediction:', prediction[0])
print('Actual:', test_labels[0])

Review: <START> inspired by hitchcock's strangers on a train concept of two men <UNK> murders in exchange for getting rid of the two people messing up their lives throw <UNK> from the train is an original and very inventive comedy take on the idea it's a credit to danny <UNK> that he both wrote and starred in this minor comedy gem br br anne <UNK> is the mother who <UNK> the film's title and it's understandable why she gets under the skin of danny <UNK> with her sharp tongue and relentlessly putting him down for any minor <UNK> billy crystal is the writer who's wife has stolen his book idea and is now being <UNK> as a great new author even appearing on the oprah show to in <UNK> he should be enjoying thus <UNK> gets the idea of <UNK> murders to rid themselves of these <UNK> factors br br of course everything and anything can happen when writer carl <UNK> lets his imagination <UNK> with <UNK> ideas for how the plot develops and it's amusing all the way through providing plenty of laughs